In [1]:
import os
import pandas as pd
import numpy as np
import json

In [2]:
#A function to read json files from ipl_json folder
path_json = '../Inputs/ipl_json'

def get_info_csv(file):
    pre_match_info_li = []
    player = []
    with open(file, 'r') as f:
        # read the header
        # iterate over the remaining lines
        for line_num, line in enumerate(f, start=1):
            fields = line.strip().split(',')
            
            if (line_num >= 2) and (line_num <= 21) and (line_num != 10):
                print(line_num)
                print(fields)
                pre_match_info_li.append(fields[1:])
            elif line_num == 10:
                pre_match_info_li.append([fields[1], fields[2]])
            elif (line_num > 21) and (line_num <= 43):
                # print(fields[2:])
                player.append(fields[2:])
            else:
                continue
        pre_match_info = pd.DataFrame(pre_match_info_li, columns=['data', 'info'])
        players = pd.DataFrame(player, columns=['Team','player'])
    return (pre_match_info, players)

def get_info_json(jsondata):
    keys = ['event', 'match_type', 'officials', 'outcome', 'season', 'teams', 'players', 'toss', 'venue']
    match_info, player_info = {}, []
    for key in keys:
        # print(f"{key}: {jsondata['info'][key]}")
        if key == 'event':
            match_info[key] = jsondata['info'][key]['name']
        elif key == 'match_type':
            match_info[key] = jsondata['info'][key]
        elif key == 'officials':
            match_info['umpire_1'] = jsondata['info'][key]['umpires'][0]
            match_info['umpire_2'] = jsondata['info'][key]['umpires'][1]
        elif key == 'outcome':
            try:
                match_info['winner'] = jsondata['info'][key]['winner']
                try:
                    match_info['win_by_runs'] = jsondata['info'][key]['by']['runs']
                except:
                    match_info['win_by_wickets'] = jsondata['info'][key]['by']['wickets']
            except:
                match_info['winner'] = 'Draw'

        elif key == 'teams':
            match_info['team_1'] = jsondata['info'][key][0]
            match_info['team_2'] = jsondata['info'][key][1]
        elif key == 'players':
            player_info = pd.DataFrame(jsondata['info'][key])
        elif key == 'toss':
            match_info['toss_winner'] = jsondata['info'][key]['winner']
            match_info['toss_decision'] = jsondata['info'][key]['decision']
        elif key == 'venue':
            match_info[key] = jsondata['info'][key]
    return match_info, player_info

def read_json(path_json):
    json_files = [pos_json for pos_json in os.listdir(path_json) if pos_json.endswith('.json')]
    matchdf, playerdf = [], []
    for file in json_files:
        print(file[:-5])
        with open(os.path.join(path_json, file)) as json_file:
            json_data = json.load(json_file)
            match_info, player_info = get_info(json_data)
        matchdf.append(match_info)
        playerdf.append(player_info)
    return (matchdf, playerdf)


In [3]:
df = pd.read_csv('../Inputs/all_matches.csv', index_col=None, header=0,low_memory=False)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
match_id,630668.0,979041.320776,347858.388412,211028.0,598061.0,1144162.0,1275047.0,1362242.0
innings,630668.0,1.475077,0.501660,1.0,1.0,1.0,2.0,6.0
ball,630668.0,9.403794,5.658149,0.1,4.4,9.3,14.3,19.9
runs_off_bat,630668.0,1.179968,1.573854,0.0,0.0,1.0,1.0,7.0
extras,630668.0,0.071760,0.352025,0.0,0.0,0.0,0.0,7.0
wides,21270.0,1.207757,0.777127,1.0,1.0,1.0,1.0,5.0
noballs,2689.0,1.035701,0.335506,1.0,1.0,1.0,1.0,5.0
byes,2136.0,1.835674,1.269959,1.0,1.0,1.0,3.0,5.0
legbyes,9941.0,1.289307,0.814973,1.0,1.0,1.0,1.0,5.0
penalty,9.0,5.000000,0.000000,5.0,5.0,5.0,5.0,5.0


In [26]:
def get_stats(df, till_date, typ='bowler'):
    # Get the data till the given date
    df['start_date'] = pd.to_datetime(df['start_date'])
    df = df[df['start_date'] < till_date]
    # print(df.shape)

    # Get the number of runs scored by each batsman
    runs = df.groupby(typ)['runs_off_bat'].value_counts()
    runs = runs.unstack(level=1)
    runs = runs.dropna(axis=1, thresh=runs.shape[0]*0.1)
    runs = runs.fillna(0)
    runs.columns = ['0_runs', '1_runs', '2_runs', '3_runs', '4_runs', '6_runs']
    # print(runs.head(2))

    # Get the number of balls faced by each batsman
    balls = df.groupby(typ)['ball'].count()
    balls = pd.DataFrame(balls)
    # print(balls.head(2))

    # Get the number of wides faced by each batsman
    df.loc[:,'wides'] = df['wides'].fillna(0).tolist()
    df['wides'] = [0 if x == 0 else 1 for x in df['wides']]
    wides = df.groupby(typ)['wides'].sum()
    wides = pd.DataFrame(wides)

    # Get the number of dismissals faced by each batsman
    dismissal = df['wicket_type'].value_counts().index.tolist()
    df.loc[:,'is_wicket'] = np.where(df['wicket_type'].isin(dismissal), 1, 0).tolist()
    dismissals = df.groupby(typ)['is_wicket'].sum()
    dismissals = pd.DataFrame(dismissals)
    dismissals = dismissals.rename(columns={'is_wicket':'num_dismissals'})

    # Add the number of balls faced and the number of wides faced to get the total number of balls faced
    balls['total_balls'] = balls['ball']+wides['wides']
    runs['0_runs'] = runs['0_runs'].subtract(dismissals['num_dismissals'])

    # Merge all the dataframes
    stats = pd.concat([runs, dismissals, wides], axis=1)

    # Divide all the columns by the number of balls faced by each batsman
    stats = stats.div(balls['total_balls'], axis=0)
    
    return stats.reset_index()


In [27]:
get_stats(df, till_date='2017-05-21', typ='bowler')


(281533, 23)
                0_runs  1_runs  2_runs  3_runs  4_runs  6_runs
bowler                                                        
A Ashish Reddy    89.0   110.0    23.0     2.0    26.0    20.0
A Bhattarai       11.0     7.0     0.0     0.0     1.0     0.0
                ball
bowler              
A Ashish Reddy   270
A Bhattarai       19


,bowler,0_runs,1_runs,2_runs,3_runs,4_runs,6_runs,num_dismissals,wides
0,A Ashish Reddy,0.253623,0.398551,0.083333,0.007246,0.094203,0.072464,0.068841,0.021739
1,A Bhattarai,0.450000,0.350000,0.000000,0.000000,0.050000,0.000000,0.100000,0.050000
2,A Chandila,0.401709,0.388889,0.042735,0.004274,0.072650,0.042735,0.047009,0.000000
3,A Choudhary,0.385965,0.271930,0.078947,0.000000,0.114035,0.052632,0.043860,0.052632
4,A Dananjaya,0.285714,0.440476,0.095238,0.000000,0.035714,0.059524,0.083333,0.000000
...,...,...,...,...,...,...,...,...,...
827,Zahoor Khan,0.272727,0.484848,0.075758,0.015152,0.015152,0.045455,0.045455,0.045455
828,Zamir Khan,0.285714,0.428571,0.142857,0.000000,0.035714,0.000000,0.035714,0.071429
829,Zeeshan Maqsood,0.343137,0.421569,0.073529,0.009804,0.049020,0.053922,0.049020,0.000000
830,Ziaur Rahman,0.192308,0.487179,0.051282,0.000000,0.153846,0.064103,0.038462,0.012821


In [8]:
df['start_date'] = pd.to_datetime(df['start_date'])

In [9]:
df['start_date'].value_counts()

2022-07-15    1848
2021-10-17    1830
2021-10-20    1749
2021-10-21    1708
2021-10-19    1693
              ... 
2014-08-27      29
2022-06-19      22
2020-11-30      16
2021-08-01       8
2013-06-27       2
Name: start_date, Length: 1716, dtype: int64

In [10]:
df = df[df['start_date'] < '2017-05-21']
df.shape

(281533, 22)

In [11]:
runs_given = df.groupby('bowler')['runs_off_bat'].value_counts()
runs_given = runs_given.unstack(level=1)
runs_given.dropna(axis=1, thresh=runs_given.shape[0]*0.1, inplace=True)
runs_given.fillna(0, inplace=True)
runs_given.columns = ['0_runs', '1_runs', '2_runs', '3_runs', '4_runs', '6_runs']
runs_given

,0_runs,1_runs,2_runs,3_runs,4_runs,6_runs
bowler,,,,,,
A Ashish Reddy,89.0,110.0,23.0,2.0,26.0,20.0
A Bhattarai,11.0,7.0,0.0,0.0,1.0,0.0
A Chandila,105.0,91.0,10.0,1.0,17.0,10.0
A Choudhary,49.0,31.0,9.0,0.0,13.0,6.0
A Dananjaya,31.0,37.0,8.0,0.0,3.0,5.0
...,...,...,...,...,...,...
Zahoor Khan,21.0,32.0,5.0,1.0,1.0,3.0
Zamir Khan,9.0,12.0,4.0,0.0,1.0,0.0
Zeeshan Maqsood,80.0,86.0,15.0,2.0,10.0,11.0


In [12]:
#create a new column for is_wicket which is 1 if wicket is taken and 0 if not
dismissal = df['wicket_type'].value_counts().index.tolist()
df['is_wicket'] = np.where(df['wicket_type'].isin(dismissal), 1, 0)
wickets_taken = df.groupby('bowler')['is_wicket'].sum()
wickets_taken = pd.DataFrame(wickets_taken)
wickets_taken.rename(columns={'is_wicket':'num_dismissals'}, inplace=True)
# wickets_taken.reset_index(inplace=True)
wickets_taken

,num_dismissals
bowler,
A Ashish Reddy,19
A Bhattarai,2
A Chandila,11
A Choudhary,5
A Dananjaya,7
...,...
Zahoor Khan,3
Zamir Khan,1
Zeeshan Maqsood,10


In [13]:
runs_given['0_runs'] = runs_given['0_runs'].subtract(wickets_taken['num_dismissals'])

In [14]:
balls_played = df.groupby('bowler')['ball'].count()
balls_played = pd.DataFrame(balls_played)
# balls_bowled.reset_index(inplace=True)
balls_played.sort_values(by='ball', ascending=False)

,ball
bowler,
SL Malinga,4182
Harbhajan Singh,3615
R Ashwin,3399
DJ Bravo,3188
DW Steyn,3108
...,...
KMC Bandara,6
AK Perera,6
Asghar Stanikzai,4


In [15]:
# Get the number of wides given by each bowler
df['wides'].fillna(0, inplace=True)
df['wides'] = [0 if x == 0 else 1 for x in df['wides']]
wides_given = df.groupby('bowler')['wides'].sum()
wides_given = pd.DataFrame(wides_given)
# wides_given.reset_index(inplace=True)
wides_given

,wides
bowler,
A Ashish Reddy,6
A Bhattarai,1
A Chandila,0
A Choudhary,6
A Dananjaya,0
...,...
Zahoor Khan,3
Zamir Khan,2
Zeeshan Maqsood,0


In [16]:
balls_played['total_balls'] = balls_played['ball']+wides_given['wides']
balls_played

,ball,total_balls
bowler,,
A Ashish Reddy,270,276
A Bhattarai,19,20
A Chandila,234,234
A Choudhary,108,114
A Dananjaya,84,84
...,...,...
Zahoor Khan,63,66
Zamir Khan,26,28
Zeeshan Maqsood,204,204


In [17]:
# Merge all the dataframes
bowler_stats = pd.concat([runs_given, wickets_taken, wides_given], axis=1)

# Divide all the columns by the number of balls faced by each batsman
bowler_stats = bowler_stats.div(balls_played['total_balls'], axis=0)
# batsman_stats = batsman_stats.reset_index().rename(columns={'index':'player'})

In [18]:
bowler_stats

,0_runs,1_runs,2_runs,3_runs,4_runs,6_runs,num_dismissals,wides
bowler,,,,,,,,
A Ashish Reddy,0.253623,0.398551,0.083333,0.007246,0.094203,0.072464,0.068841,0.021739
A Bhattarai,0.450000,0.350000,0.000000,0.000000,0.050000,0.000000,0.100000,0.050000
A Chandila,0.401709,0.388889,0.042735,0.004274,0.072650,0.042735,0.047009,0.000000
A Choudhary,0.385965,0.271930,0.078947,0.000000,0.114035,0.052632,0.043860,0.052632
A Dananjaya,0.285714,0.440476,0.095238,0.000000,0.035714,0.059524,0.083333,0.000000
...,...,...,...,...,...,...,...,...
Zahoor Khan,0.272727,0.484848,0.075758,0.015152,0.015152,0.045455,0.045455,0.045455
Zamir Khan,0.285714,0.428571,0.142857,0.000000,0.035714,0.000000,0.035714,0.071429
Zeeshan Maqsood,0.343137,0.421569,0.073529,0.009804,0.049020,0.053922,0.049020,0.000000


In [28]:
np.allclose(bowler_stats, get_stats(df, till_date='2017-05-21', typ='bowler').set_index('bowler'))

(281533, 23)
                0_runs  1_runs  2_runs  3_runs  4_runs  6_runs
bowler                                                        
A Ashish Reddy    89.0   110.0    23.0     2.0    26.0    20.0
A Bhattarai       11.0     7.0     0.0     0.0     1.0     0.0
                ball
bowler              
A Ashish Reddy   270
A Bhattarai       19


True